In [ ]:
import numpy as np
import gym
import random
import time
from IPython.display import clear_output

In [ ]:
#Make the environment
env = gym.make("FrozenLake-v0")
env.reset()
env.render()

In [ ]:
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

In [ ]:
#Build the Q-Table
q_table = np.zeros((state_space_size, action_space_size))
print(q_table)

In [ ]:
#Create and Initialize all the parameters needed to implement the Q-Learning algorithm
num_episodes = 10000
max_steps_per_episode = 100 #For termination, in a episode

learning_rate = 0.1
discount_rate = 0.99 # Between 0 and 1 symbol

exploration_rate = 1 # epsilon value
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001 #Change with experiment

#The above parameters are to be trained and tuned

In [ ]:
#Create a list to store the rewards in each episode
#To see how the game scores change everytime
rewards_all_episodes = []

# Q-learning algorithm
for episode in range(num_episodes):
    #Resetting to starting state for each episode
    state = env.reset()
    #To make sure the episode is finished
    done = False 
    rewards_current_episode = 0 #Keep track of reward in each episode

    for step in range(max_steps_per_episode): 
        # Exploration-exploitation trade-off
        exploration_rate_threshold = random.uniform(0, 1) #This is needed to whether know that our agent will eplore
                                                          # or exploit the environment
        if exploration_rate_threshold > exploration_rate: 
            # exploitation
            action = np.argmax(q_table[state,:])
        else:
            # Exploration
            action = env.action_space.sample()      

        # After the action is chosen, we then execute the action
        new_state, reward, done, info = env.step(action) 
        
        #1. new_state 
        #2. reward for the action 
        #3. whether or not action ended our episode 
        #4. diagnostic information of the environment

        # Update Q-table for Q(s,a) as per the formula (weighted sum of old value and learned value)
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + \
                                learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))

        state = new_state #Current state is the new state
        rewards_current_episode += reward #add the cumulative reward

        #check if the episode is over or not
        if done == True: 
          break

    # Exploration rate decay
    exploration_rate = min_exploration_rate + \
                       (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)
                       
    #appends the rewards from the current episode               
    rewards_all_episodes.append(rewards_current_episode)

In [ ]:
# After all episodes are completed, we calculate average reward per 1000 episodes
# Calculate and print the average reward per thousand episodes
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/1000)
count = 1000

print("********Average reward per thousand episodes********\n")
for r in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000

In [ ]:
# Print updated Q-table
print("\n\n********Q-table********\n")
print(q_table)

In [ ]:
for episode in range(3):
    state = env.reset()
    done = False
    print("*****EPISODE ", episode+1, "*****\n\n\n\n")
    time.sleep(1)

    for step in range(max_steps_per_episode):        
        clear_output(wait=True)
        env.render() # render the current state of the environment to the display so 
                     # that we can visualize the gamegrid and agent gale play
        time.sleep(0.3)

        action = np.argmax(q_table[state,:])        
        new_state, reward, done, info = env.step(action)

        # If the episode is completed then we render the environment to see where the agent ended in the
        # environment by taking that action
        if done:
            clear_output(wait=True)
            env.render()
            if reward == 1:
                print("****You reached the goal!****")
                time.sleep(3)
            else:
                print("****You fell through a hole!****")
                time.sleep(3)
            clear_output(wait=True)
            break
        
        state = new_state

env.close()